## Run Inference on all deployed endpoints: Various combinations of payloads, concurrency levels, model configurations
---------------------
*This notebook works best with the conda_python3 kernel on a ml.t3.medium machine*.

#### This step of our solution design includes running inferences on all deployed model endpoints (with different configurations, concurrency levels and payload sizes). This notebook runs inferences in a manner that is calls endpoints concurrently and asychronously to generate responses and record metrics. Here are some of the key components:

- **Accessing the deployed endpoints**, creating a predictor object for these endpoints to call them during inference time.

- **Functions to define metrics**: This notebook sets stage for metrics to be recorded during the time of invocation of all these models for benchmarking purposes.

- **Running Actual Inferences**: Once the metrics are defined, we set a blocker function that is responsible for creating inference on a single payload called get_inference. We then run a series of asynchronous functions that can be viewed in the code (link above), to create asychronous inferefences on the deployed models. The way we send requests are by creating combinations: this means creating combinations of payloads of different sizes that can be viewed in the config.yml file, with different concurrency levels (in this case we first go through all patches of payloads with a concurrency level of 1, then 2, and then 4). You can set this to your desired value.

In [130]:
## auto reload all of the changes made in the config/globals.py file 
%load_ext autoreload
%autoreload 2
!touch globals.py

CONFIG_FILE=configs/config-mistral-7b-tgi-g5.yml
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Import all of the necessary libraries below to run this notebook

In [131]:
import glob
import time
import json
import io
import copy
import boto3
import asyncio
import logging
import itertools
import sagemaker
import pandas as pd
from globals import *
from datetime import datetime
from transformers import AutoTokenizer
from sagemaker.predictor import Predictor
from utils import load_config, count_tokens, write_to_s3, read_from_s3, get_s3_object
from sagemaker.serializers import JSONSerializer
from typing import Dict, List, Optional, Tuple, Union

CONFIG_FILE=configs/config-mistral-7b-tgi-g5.yml


#### Pygmentize globals.py to view and use any of the globally initialized variables 

In [132]:
# global constants
!pygmentize globals.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


import os
import yaml
from enum import Enum
from pathlib import Path
import boto3
from datetime import datetime

CONFIG_FILEPATH_FILE: str = "config_filepath.txt"

# S3 client initialization
s3_client = boto3.client('s3')

CONFIG_FILE: str = Path(CONFIG_FILEPATH_FILE).read_text()
print(f"CONFIG_FILE={CONFIG_FILE}")
with open(CONFIG_FILE, 'r') as file:
    config = yaml.safe_load(file)

DATA_DIR: str = config['dir_paths']['data_prefix']
PROMPTS_DIR = os.path.join(DATA_DIR, config['dir_paths']['prompts_prefix'])

## --------------------- Metrics directory based on date and time ---------------------------
current_time = datetime.now()
formatted_time = current_time.strftime("%Y/%m/%d/%H/%M")
METRICS_DIR = f"{config['dir_paths']['data_prefix']}/metrics/{formatted_time}/{config['general']['name']}"


METRICS_PER_INFERENCE_DIR  = os.path.join(METRICS_DIR, "per_inference")
METRICS_PER_CHUNK_DIR  = os.path.join(METRICS_DIR, "per_chunk")


## --------------------- Models directory based on date

In [133]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

#### Load the Config.yml file that contains information that is used across this benchmarking environment

In [134]:
config = load_config(CONFIG_FILE)
logger.info(json.dumps(config, indent=2))

[2024-01-29 19:52:45,598] p58222 {635462509.py:2} INFO - {
  "general": {
    "name": "mistral-7b-tgi-g5-v1",
    "model_name": "mistral7b"
  },
  "aws": {
    "region": "us-east-1",
    "sagemaker_execution_role": "arn:aws:iam::218208277580:role/service-role/AmazonSageMaker-ExecutionRole-20230807T175994",
    "bucket": "fmbt"
  },
  "dir_paths": {
    "data_prefix": "data",
    "source_data_prefix": "source_data",
    "prompt_template_dir": "prompt_template",
    "prompt_template_file": "prompt_template.txt",
    "tokenizer_prefix": "tokenizer",
    "scripts_prefix": "scripts",
    "prompts_prefix": "prompts",
    "all_prompts_file": "all_prompts.csv",
    "metrics_dir": "metrics_{datetime}",
    "models_dir": "models_{datetime}"
  },
  "datasets": [
    {
      "language": "en",
      "min_length_in_tokens": 1,
      "max_length_in_tokens": 500,
      "payload_file": "payload_{lang}_{min}-{max}.jsonl"
    },
    {
      "language": "en",
      "min_length_in_tokens": 500,
      "max_

In [135]:
date_time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

In [136]:
## getting access to the s3 bucket where endpoints.json for different models resides
s3_client = boto3.client('s3')

### Access the deployed model endpoints from the endpoints.json file 

In [137]:
with open('endpoint_path.txt', 'r') as file:
    ENDPOINT_S3_PATH = file.read().strip()
    logger.info(f"endpoint path -> {ENDPOINT_S3_PATH}")

endpoint_info_list = json.loads(get_s3_object(ENDPOINT_S3_PATH))
logger.info(f"found information for {len(endpoint_info_list)} endpoints")
logger.info(json.dumps(endpoint_info_list, indent=2))

[2024-01-29 19:52:45,733] p58222 {429287404.py:3} INFO - endpoint path -> s3://fmbt/data/models/2024/01/29/19/38/mistral-7b-tgi-g5-v1/lmistral7b-g5-2xlarge-1706575144_endpoints.json


[2024-01-29 19:52:45,884] p58222 {429287404.py:6} INFO - found information for 1 endpoints
[2024-01-29 19:52:45,884] p58222 {429287404.py:7} INFO - [
  {
    "experiment_name": "mistral-7b-g5-huggingface-pytorch-tgi-inference-2.0.1-tgi1.1.0",
    "endpoint": {
      "EndpointName": "lmistral7b-g5-2xlarge-1706575144",
      "EndpointArn": "arn:aws:sagemaker:us-east-1:218208277580:endpoint/lmistral7b-g5-2xlarge-1706575144",
      "EndpointConfigName": "lmistral7b-g5-2xlarge-1706575144",
      "ProductionVariants": [
        {
          "VariantName": "AllTraffic",
          "DeployedImages": [
            {
              "SpecifiedImage": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi1.1.0-gpu-py39-cu118-ubuntu20.04",
              "ResolvedImage": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference@sha256:2739b630b95d8a95e6b4665e66d8243dd43b99c4fdb865feff13aab9c1da06eb",
              "ResolutionTime": "2024-01-29 19

In [138]:
# List down the endpoint names that have been deployed
endpoint_name_list = [e['endpoint']['EndpointName'] for e in endpoint_info_list]
logger.info(f"there are {len(endpoint_name_list)} deployed endpoint(s), endpoint_name_list->{endpoint_name_list}")

[2024-01-29 19:52:45,918] p58222 {1455142584.py:3} INFO - there are 1 deployed endpoint(s), endpoint_name_list->['lmistral7b-g5-2xlarge-1706575144']


### Creating predictor objects from the deployed endpoints

In [139]:
# create predictor objects

## create a sagemaker predictor for these endpoints
def create_predictor(endpoint_name: str) -> Optional[sagemaker.base_predictor.Predictor]:
    # Create a SageMaker Predictor object
    predictor = Predictor(
        endpoint_name=endpoint_name,
        sagemaker_session=sagemaker.Session(),
        serializer=JSONSerializer()
    )
    return predictor

## Display the list of predictor objects that have been deployed ready for inferencing from
predictor_list: List = [create_predictor(ep) for ep in endpoint_name_list]
logger.info(predictor_list)

[2024-01-29 19:52:46,458] p58222 {3970465137.py:15} INFO - [<sagemaker.base_predictor.Predictor object at 0x1101fcb10>]


### Creating functions to define and calculate metrics during the time of invocations

In [140]:
def safe_sum(l: List) -> Union[int, float]:
    return sum(filter(None, l))

def safe_div(n: Union[int, float], d: Union[int, float]) -> Optional[Union[int, float]]:
    return n/d if d else None

## Represents the function to calculate all of the metrics at the time of inference
def calculate_metrics(responses, chunk, elapsed_async, experiment_name, concurrency, payload_file) -> Dict:
    
    ## calculate errors based on the completion status of the inference prompt
    errors = [r for r in responses if r['completion'] is None]
    
    ## Calculate the difference as the successes 
    successes = len(chunk) - len(errors)
    
    ## Count all of the prompts token count during inference
    all_prompts_token_count = safe_sum([r['prompt_tokens'] for r in responses])
    prompt_token_throughput = round(all_prompts_token_count / elapsed_async, 2)
    prompt_token_count_mean = safe_div(all_prompts_token_count, successes)
    all_completions_token_count = safe_sum([r['completion_tokens'] for r in responses])
    completion_token_throughput = round(all_completions_token_count / elapsed_async, 2)
    completion_token_count_mean = safe_div(all_completions_token_count, successes)
    transactions_per_second = round(successes / elapsed_async, 2)
    transactions_per_minute = int(transactions_per_second * 60)
    
    ## calculate the latency mean utilizing the safe_sum function defined above
    latency_mean = safe_div(safe_sum([r['latency'] for r in responses]), successes)
    
    ## Function returns all these values at the time of the invocations
    return {
        'experiment_name': experiment_name,
        'concurrency': concurrency,
        'payload_file': payload_file,
        'errors': errors,
        'successes': successes,
        'error_rate': len(errors)/len(chunk),
        'all_prompts_token_count': all_prompts_token_count,
        'prompt_token_count_mean': prompt_token_count_mean,
        'prompt_token_throughput': prompt_token_throughput,
        'all_completions_token_count': all_completions_token_count,
        'completion_token_count_mean': completion_token_count_mean,
        'completion_token_throughput': completion_token_throughput,
        'transactions': len(chunk),
        'transactions_per_second': transactions_per_second,
        'transactions_per_minute': transactions_per_minute,
        'latency_mean': latency_mean
    }

### Set a blocker function and a series of asynchronous concurrent model prompt invocations

In [141]:
def set_metrics(endpoint_name=None,
                    prompt=None,
                    inference_params=None,
                    completion=None,
                    prompt_tokens=None,
                    completion_tokens=None,
                    latency=None) -> Dict:
    return dict(endpoint_name=endpoint_name,                
                prompt=prompt,
                **inference_params,
                completion=completion,
                prompt_tokens=prompt_tokens,
                completion_tokens=completion_tokens,
                latency=latency)

def get_inference(predictor, payload) -> Dict:
    
    smr_client = boto3.client("sagemaker-runtime")
    latency = 0

    try:
        prompt_tokens = count_tokens(payload['inputs'])
        logger.info(f"get_inference, endpoint={predictor.endpoint_name}, prompt_tokens={prompt_tokens}")

        # get inference
        st = time.perf_counter()        
        response = predictor.predict(payload)        
        latency = time.perf_counter() - st

        if isinstance(response, bytes):
            response = response.decode('utf-8')
        response_json = json.loads(response)
        if isinstance(response_json, list):
            response_json = response_json[0]

        completion = response_json.get("generated_text", "")
        completion_tokens = count_tokens(completion)

        # Set metrics and logging for both cases
        response = set_metrics(predictor.endpoint_name,
                               payload['inputs'],
                               payload['parameters'],
                               completion,
                               prompt_tokens,
                               completion_tokens,
                               latency)
        # logger.info(f"get_inference, done, endpoint={predictor.endpoint_name}, response={json.dumps(response, indent=2)}, latency={latency:.2f}")
        logger.info(f"get_inference, done, endpoint={predictor.endpoint_name}, completion_tokens={completion_tokens}, latency={latency:.2f}")
    except Exception as e:
        print(f"error occurred with {predictor.endpoint_name}, exception={str(e)}")
        response = set_metrics(predictor.endpoint_name,
                               payload['inputs'],
                               payload['parameters'],
                               None,
                               prompt_tokens,
                               None,
                               None)

    return response

### Setting a series of asynchronous functions to invoke and run inferences concurrently and asynchronously

In [142]:
## Represents a function to start invoking models in separate thread asynchronously for the blocker function
async def async_get_inference(predictor, payload: Dict) -> Dict:
    return await asyncio.to_thread(get_inference, predictor, payload)

## Gathers all of the tasks and sets of the concurrent calling of the asychronous invocations
async def async_get_all_inferences(predictor, payload_list: List) -> List:
    return await asyncio.gather(*[async_get_inference(predictor, payload) for payload in payload_list])

In [143]:
## This function runs the asychronous function series above together for different experiments and concurrency levels.
async def run_inferences(predictor: sagemaker.base_predictor.Predictor, chunk: List, experiment: Dict, concurrency: int, payload_file: str) -> Tuple[List, Dict]:
    logger.info(f"Processing chunk with concurrency={concurrency}")
    s = time.perf_counter()
    responses = await async_get_all_inferences(predictor, chunk)
    elapsed_async = time.perf_counter() - s

    # Add more metadata about this experiment
    for r in responses:
        r['experiment_name'] = experiment['name']
        r['concurrency'] = concurrency

    metrics = calculate_metrics(responses, chunk, elapsed_async, experiment['name'], concurrency, payload_file)
    return responses, metrics

In [144]:
## Function to create the predictors from the experiment we are iterating over
def create_predictor_for_experiment(experiment: str, config: Dict, endpoint_info_list: List) -> Optional[sagemaker.base_predictor.Predictor]:

    ## Here, we set the index and then iterate through the experiments
    e_idx = config['experiments'].index(experiment) + 1

    ## Iterate through the endpoint information to fetch the endpoint name
    ep_info = [e for e in endpoint_info_list if e['experiment_name'] == experiment['name']]
    if not ep_info:
        logger.error(f"endpoint for experiment={experiment['name']} not found, skipping")
        return None
    ep_name = ep_info[0]['endpoint']['EndpointName']
    logger.info(f"experiment={e_idx}, name={experiment['name']}, ep_name={ep_name}")

    # create a predictor from each endpoint in experiments
    return create_predictor(ep_name)

In [145]:
## Here, we will process combinations of concurrency levels, the payload files and then loop through the 
## different combinations to make payloads splitted in terms of the concurrency metric and how we can run 
## it and make inference

def create_payload_dict(jline: str, experiment: Dict) -> Dict:
    payload: Dict = json.loads(jline)
    if experiment.get('remove_truncate', False) is True:
        if payload['parameters'].get('truncate'):
            del payload['parameters']['truncate']
    return payload
    
    
def create_combinations(experiment: Dict) -> List[Tuple]:
    combinations_data = []

    # Repeat for each concurrency level
    combinations = list(itertools.product(experiment['concurrency_levels'], experiment['payload_files']))
    logger.info(f"there are {len(combinations)} combinations of {combinations} to run")

    for concurrency, payload_file in combinations:
        # Construct the full S3 file path
        s3_file_path = os.path.join(PROMPTS_DIR, payload_file)
        logger.info(f"s3 path where the payload files are being read from -> {s3_file_path}")

        # Read the payload file from S3
        try:
            response = s3_client.get_object(Bucket=config['aws']['bucket'], Key=s3_file_path)
            payload_file_content = response['Body'].read().decode('utf-8')

            # Create a payload list by processing each line
            payload_list = [create_payload_dict(jline, experiment) for jline in payload_file_content.splitlines()]
            logger.info(f"read from s3://{config['aws']['bucket']}/{s3_file_path}, contains {len(payload_list)} lines")

        except Exception as e:
            logger.error(f"Error reading file from S3: {e}")
            continue

        logger.info(f"creating combinations for concurrency={concurrency}, payload_file={payload_file}, payload_list length={len(payload_list)}")
        
        n = concurrency
        
        if len(payload_list) < n:
            elements_to_add = n - len(payload_list)
            element_to_replicate = payload_list[0]
            # payload_list = payload_list.extend([element_to_replicate]*elements_to_add)
            payload_list.extend([element_to_replicate]*elements_to_add)
            
        # Split the original list into sublists which contain the number of requests we want to send concurrently        
        payload_list_splitted = [payload_list[i * n:(i + 1) * n] for i in range((len(payload_list) + n - 1) // n )]  
        
        for p in payload_list_splitted:
            if len(p) < n:
                elements_to_add = n - len(p)
                element_to_replicate = p[0]
                # p = p.extend([element_to_replicate]*elements_to_add)
                p.extend([element_to_replicate]*elements_to_add)
            

        # Only keep lists that have at least concurrency number of elements
        len_before = len(payload_list_splitted)
        payload_list_splitted = [p for p in payload_list_splitted if len(p) == concurrency]
        logger.info(f"after only retaining chunks of length {concurrency}, we have {len(payload_list_splitted)} chunks, previously we had {len_before} chunks")
        combinations_data.append((concurrency, payload_file, payload_list_splitted))
    logger.info(f"there are {len(combinations)} for {experiment}")
    return combinations_data

# process_combinations(experiment, predictor, PROMPTS_DIR)

In [146]:
# for each experiment
#   - for each endpoint and concurrency in an experiment

def clear_dir(dir_path: str):
    files = glob.glob(os.path.join(dir_path, "*"))
    for f in files:
        os.remove(f)

_ = list(map(clear_dir, [METRICS_PER_INFERENCE_DIR, METRICS_PER_CHUNK_DIR]))

num_experiments: int = len(config['experiments'])
for e_idx, experiment in enumerate(config['experiments']):
    e_idx += 1  # Increment experiment index
    # Call do_experiment function to create the predictor object
 
    predictor = create_predictor_for_experiment(experiment, config, endpoint_info_list)
    if predictor is None:
        logger.error(f"predictor could not be created for experiment={experiment}, moving to next...")
        continue

    # Process combinations of concurrency levels and payload files
    combination_data = create_combinations(experiment)

    for concurrency, payload_file, split_payload in combination_data:
        for chunk_index, chunk in enumerate(split_payload):
            logger.info(f"e_idx={e_idx}/{num_experiments}, chunk_index={chunk_index+1}/{len(split_payload)}")

            # Process each chunk and calculate metrics
            responses, metrics = await run_inferences(predictor, chunk, experiment, concurrency, payload_file)
            if metrics:
                # Convert metrics to JSON string
                metrics_json = json.dumps(metrics, indent=2)
                # Define S3 file path
                metrics_file_name = f"{time.time()}.json"
                metrics_s3_path = os.path.join(METRICS_PER_CHUNK_DIR, metrics_file_name)
                # Write to S3
                write_to_s3(metrics_json, config['aws']['bucket'], "", METRICS_PER_CHUNK_DIR, metrics_file_name)

            if responses:
                for r in responses:
                    # Convert response to JSON string
                    response_json = json.dumps(r, indent=2)
                    # Define S3 file path
                    response_file_name = f"{time.time()}.json"
                    response_s3_path = os.path.join(METRICS_PER_INFERENCE_DIR, response_file_name)
                    # Write to S3
                    write_to_s3(response_json, config['aws']['bucket'], "", METRICS_PER_INFERENCE_DIR, response_file_name)

    logger.info(f"experiment={e_idx}/{num_experiments}, name={experiment['name']}, done")

[2024-01-29 19:52:51,130] p58222 {663335596.py:13} INFO - experiment=1, name=mistral-7b-g5-huggingface-pytorch-tgi-inference-2.0.1-tgi1.1.0, ep_name=lmistral7b-g5-2xlarge-1706575144
[2024-01-29 19:52:51,154] p58222 {2135215762.py:18} INFO - there are 3 combinations of [(1, 'payload_en_1-500.jsonl'), (1, 'payload_en_500-1000.jsonl'), (1, 'payload_en_1000-2000.jsonl')] to run
[2024-01-29 19:52:51,155] p58222 {2135215762.py:23} INFO - s3 path where the payload files are being read from -> data/prompts/payload_en_1-500.jsonl


[2024-01-29 19:52:51,336] p58222 {2135215762.py:32} INFO - read from s3://fmbt/data/prompts/payload_en_1-500.jsonl, contains 1 lines
[2024-01-29 19:52:51,336] p58222 {2135215762.py:38} INFO - creating combinations for concurrency=1, payload_file=payload_en_1-500.jsonl, payload_list length=1
[2024-01-29 19:52:51,336] p58222 {2135215762.py:62} INFO - after only retaining chunks of length 1, we have 1 chunks, previously we had 1 chunks
[2024-01-29 19:52:51,336] p58222 {2135215762.py:23} INFO - s3 path where the payload files are being read from -> data/prompts/payload_en_500-1000.jsonl
[2024-01-29 19:52:51,415] p58222 {2135215762.py:32} INFO - read from s3://fmbt/data/prompts/payload_en_500-1000.jsonl, contains 1 lines
[2024-01-29 19:52:51,416] p58222 {2135215762.py:38} INFO - creating combinations for concurrency=1, payload_file=payload_en_500-1000.jsonl, payload_list length=1
[2024-01-29 19:52:51,417] p58222 {2135215762.py:62} INFO - after only retaining chunks of length 1, we have 1 ch

In [155]:
# Function to list files in S3 bucket with a specific prefix
def list_s3_files(bucket, prefix, suffix='.json'):
    response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    return [item['Key'] for item in response.get('Contents', []) if item['Key'].endswith(suffix)]

# List .json files in the specified S3 directory
s3_files = list_s3_files(config['aws']['bucket'], METRICS_PER_INFERENCE_DIR)

# Read and parse each JSON file from S3
json_list = []
for file_key in s3_files:
    response = s3_client.get_object(Bucket=config['aws']['bucket'], Key=file_key)
    file_content = response['Body'].read().decode('utf-8')
    json_obj = json.loads(file_content)
    json_list.append(json_obj)

# Create DataFrame
df_responses = pd.DataFrame(json_list)
logger.info(f"created dataframe of shape {df_responses.shape} from all responses")
df_responses.head()


[2024-01-29 19:54:15,858] p58222 {2402469004.py:19} INFO - created dataframe of shape (17, 14) from all responses


,endpoint_name,prompt,do_sample,temperature,top_p,top_k,max_new_tokens,truncate,completion,prompt_tokens,completion_tokens,latency,experiment_name,concurrency
0,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,304,\n\n```\nThe genus Sinofranchetia and Staunton...,304,98,3.670313,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1
1,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,980,Both WAGS Atlanta and WAGS are reality televi...,980,100,3.959870,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1
2,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1339,\n\n```\n```\n```\n```\n```\n```\n```\n```\n``...,1339,69,4.075735,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1
3,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1932,Altuğ Çelikbilek is younger.\n\nQuestion: Who...,1932,97,4.431883,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1
4,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1154,\n\n```\nPassage 1:\nGepiș\nThe Gepiș is a rig...,1154,101,3.965733,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1


In [156]:
df_responses[df_responses.endpoint_name.str.contains("g5")]

,endpoint_name,prompt,do_sample,temperature,top_p,top_k,max_new_tokens,truncate,completion,prompt_tokens,completion_tokens,latency,experiment_name,concurrency
0,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,304,\n\n```\nThe genus Sinofranchetia and Staunton...,304,98,3.670313,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1
1,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,980,Both WAGS Atlanta and WAGS are reality televi...,980,100,3.959870,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1
2,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1339,\n\n```\n```\n```\n```\n```\n```\n```\n```\n``...,1339,69,4.075735,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1
3,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1932,Altuğ Çelikbilek is younger.\n\nQuestion: Who...,1932,97,4.431883,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1
4,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1154,\n\n```\nPassage 1:\nGepiș\nThe Gepiș is a rig...,1154,101,3.965733,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1
5,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1646,\n\n```\nBoth Howard Lake (British Columbia) a...,1646,38,1.952168,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1
6,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1397,\n\n```\nPassage 1:\nPuka Rumi\nPuka Rumi (Que...,1397,99,4.104989,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1
7,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1746,\n\n```\nThe date of death of Parimala Nagappa...,1746,24,1.397450,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1
8,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1373,\n\n```\n```\n```\n```,1373,11,0.871415,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1
9,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1598,\n\nJavier Frana and Thomaz Koch both share th...,1598,98,4.252575,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1


In [157]:
def list_s3_files(bucket, prefix, suffix='.json'):
    response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    logger.info(f"files recieved from s3 for per inference request --> {response}")
    return [item['Key'] for item in response.get('Contents', []) if item['Key'].endswith(suffix)]

# List .json files in the specified S3 directory
s3_files = list_s3_files(config['aws']['bucket'], METRICS_PER_CHUNK_DIR)

# Read and parse each JSON file from S3
json_list = []
for file_key in s3_files:
    response = s3_client.get_object(Bucket=config['aws']['bucket'], Key=file_key)
    file_content = response['Body'].read().decode('utf-8')
    json_obj = json.loads(file_content)
    json_list.append(json_obj)

# Create DataFrame
df_metrics = pd.DataFrame(json_list)
logger.info(f"created dataframe of shape {df_metrics.shape} from all responses")
df_metrics.head()

[2024-01-29 19:54:15,967] p58222 {4027816906.py:3} INFO - files recieved from s3 for per inference request --> {'ResponseMetadata': {'RequestId': 'ECDE4NY7KANS943Q', 'HostId': 'iHlrEBeeaTrKJ5nbmuK5a1a/JrxK++SbS4+ynU6r2mAkIibEfvmhHN8UAZgbxIm3A01j3C7BgzE=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'iHlrEBeeaTrKJ5nbmuK5a1a/JrxK++SbS4+ynU6r2mAkIibEfvmhHN8UAZgbxIm3A01j3C7BgzE=', 'x-amz-request-id': 'ECDE4NY7KANS943Q', 'date': 'Tue, 30 Jan 2024 00:54:16 GMT', 'x-amz-bucket-region': 'us-east-1', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'IsTruncated': False, 'Contents': [{'Key': 'data/metrics/2024/01/29/19/52/mistral-7b-tgi-g5-v1/per_chunk/1706575975.205216.json', 'LastModified': datetime.datetime(2024, 1, 30, 0, 52, 56, tzinfo=tzutc()), 'ETag': '"bbebd616e79a28224c8ca72dcf81a8fd"', 'Size': 557, 'StorageClass': 'STANDARD'}, {'Key': 'data/metrics/2024/01/29/19/52/mistral-7b-tgi-g5-v1/per_chunk/1706575979.4627352.j

[2024-01-29 19:54:16,523] p58222 {4027816906.py:19} INFO - created dataframe of shape (17, 16) from all responses


,experiment_name,concurrency,payload_file,errors,successes,error_rate,all_prompts_token_count,prompt_token_count_mean,prompt_token_throughput,all_completions_token_count,completion_token_count_mean,completion_token_throughput,transactions,transactions_per_second,transactions_per_minute,latency_mean
0,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1,payload_en_1-500.jsonl,[],1,0.0,304,304.0,82.66,98,98.0,26.65,1,0.27,16,3.670313
1,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1,payload_en_500-1000.jsonl,[],1,0.0,980,980.0,247.10,100,100.0,25.21,1,0.25,15,3.959870
2,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1,payload_en_1000-2000.jsonl,[],1,0.0,1339,1339.0,327.49,69,69.0,16.88,1,0.24,14,4.075735
3,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1,payload_en_1000-2000.jsonl,[],1,0.0,1932,1932.0,434.65,97,97.0,21.82,1,0.22,13,4.431883
4,mistral-7b-g5-huggingface-pytorch-tgi-inferenc...,1,payload_en_1000-2000.jsonl,[],1,0.0,1154,1154.0,289.78,101,101.0,25.36,1,0.25,15,3.965733


In [158]:
df_endpoints = pd.json_normalize(endpoint_info_list)
df_endpoints['instance_type'] = df_endpoints['endpoint_config.ProductionVariants'].map(lambda x: x[0]['InstanceType'])
df_endpoints
cols_for_env = [c for c in df_endpoints.columns if 'Environment' in c]
print(cols_for_env)
cols_of_interest = ['experiment_name', 
                    'instance_type',
                    'endpoint.EndpointName',
                    'model_config.ModelName',
                    'model_config.PrimaryContainer.Image',   
                    'model_config.PrimaryContainer.ModelDataSource.S3DataSource.S3Uri']
cols_of_interest.extend(cols_for_env)

df_endpoints = df_endpoints[cols_of_interest]
df_endpoints = df_endpoints[cols_of_interest]
cols_of_interest_renamed = [c.split('.')[-1] for c in cols_of_interest]
df_endpoints.columns = cols_of_interest_renamed

# Check if 'experiment_name' column exists in both DataFrames
print("Columns in df_responses:", df_responses.columns)
print("Columns in df_endpoints:", df_endpoints.columns)

# Merge operation
df_results = pd.merge(left=df_responses, right=df_endpoints, how='left', left_on='experiment_name', right_on='experiment_name')

# Inspect the result
df_results.head()

['model_config.PrimaryContainer.Environment.ENDPOINT_SERVER_TIMEOUT', 'model_config.PrimaryContainer.Environment.HF_MODEL_ID', 'model_config.PrimaryContainer.Environment.MAX_BATCH_PREFILL_TOKENS', 'model_config.PrimaryContainer.Environment.MAX_INPUT_LENGTH', 'model_config.PrimaryContainer.Environment.MAX_TOTAL_TOKENS', 'model_config.PrimaryContainer.Environment.MODEL_CACHE_ROOT', 'model_config.PrimaryContainer.Environment.SAGEMAKER_ENV', 'model_config.PrimaryContainer.Environment.SAGEMAKER_MODEL_SERVER_WORKERS', 'model_config.PrimaryContainer.Environment.SAGEMAKER_PROGRAM', 'model_config.PrimaryContainer.Environment.SM_NUM_GPUS']
Columns in df_responses: Index(['endpoint_name', 'prompt', 'do_sample', 'temperature', 'top_p', 'top_k',
       'max_new_tokens', 'truncate', 'completion', 'prompt_tokens',
       'completion_tokens', 'latency', 'experiment_name', 'concurrency'],
      dtype='object')
Columns in df_endpoints: Index(['experiment_name', 'instance_type', 'EndpointName', 'ModelNam

,endpoint_name,prompt,do_sample,temperature,top_p,top_k,max_new_tokens,truncate,completion,prompt_tokens,...,ENDPOINT_SERVER_TIMEOUT,HF_MODEL_ID,MAX_BATCH_PREFILL_TOKENS,MAX_INPUT_LENGTH,MAX_TOTAL_TOKENS,MODEL_CACHE_ROOT,SAGEMAKER_ENV,SAGEMAKER_MODEL_SERVER_WORKERS,SAGEMAKER_PROGRAM,SM_NUM_GPUS
0,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,304,\n\n```\nThe genus Sinofranchetia and Staunton...,304,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1
1,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,980,Both WAGS Atlanta and WAGS are reality televi...,980,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1
2,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1339,\n\n```\n```\n```\n```\n```\n```\n```\n```\n``...,1339,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1
3,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1932,Altuğ Çelikbilek is younger.\n\nQuestion: Who...,1932,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1
4,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1154,\n\n```\nPassage 1:\nGepiș\nThe Gepiș is a rig...,1154,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1


In [159]:
df_results = pd.merge(left=df_responses, right=df_endpoints, how='left', left_on='experiment_name', right_on='experiment_name')
df_results.head()

,endpoint_name,prompt,do_sample,temperature,top_p,top_k,max_new_tokens,truncate,completion,prompt_tokens,...,ENDPOINT_SERVER_TIMEOUT,HF_MODEL_ID,MAX_BATCH_PREFILL_TOKENS,MAX_INPUT_LENGTH,MAX_TOTAL_TOKENS,MODEL_CACHE_ROOT,SAGEMAKER_ENV,SAGEMAKER_MODEL_SERVER_WORKERS,SAGEMAKER_PROGRAM,SM_NUM_GPUS
0,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,304,\n\n```\nThe genus Sinofranchetia and Staunton...,304,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1
1,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,980,Both WAGS Atlanta and WAGS are reality televi...,980,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1
2,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1339,\n\n```\n```\n```\n```\n```\n```\n```\n```\n``...,1339,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1
3,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1932,Altuğ Çelikbilek is younger.\n\nQuestion: Who...,1932,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1
4,lmistral7b-g5-2xlarge-1706575144,<s>[INST] <<SYS>>\nYou are an assistant for qu...,True,0.1,0.92,120,100,1154,\n\n```\nPassage 1:\nGepiș\nThe Gepiș is a rig...,1154,...,3600,/opt/ml/model,8191,8191,8192,/opt/ml/model,1,1,inference.py,1


In [160]:
# Convert df_results to CSV and write to S3
csv_buffer = io.StringIO()
df_results.to_csv(csv_buffer, index=False)
csv_data_results = csv_buffer.getvalue()
results_file_name = config['results']['per_inference_request_file'].format(datetime=date_time)
results_s3_path = os.path.join(METRICS_DIR, results_file_name)
logger.info(f"results s3 path for per inference csv --> {results_s3_path}")
write_to_s3(csv_data_results, config['aws']['bucket'], "", METRICS_DIR, results_file_name)
logger.info(f"saved results dataframe of shape={df_results.shape} in s3://{config['aws']['bucket']}/{results_s3_path}")

[2024-01-29 19:54:16,656] p58222 {1111713650.py:7} INFO - results s3 path for per inference csv --> data/metrics/2024/01/29/19/52/mistral-7b-tgi-g5-v1/per_inference_request_results.csv


[2024-01-29 19:54:17,007] p58222 {1111713650.py:9} INFO - saved results dataframe of shape=(17, 29) in s3://fmbt/data/metrics/2024/01/29/19/52/mistral-7b-tgi-g5-v1/per_inference_request_results.csv


In [161]:
METRICS_DIR
# Write the results_s3_path to a text file
with open('results_path.txt', 'w') as file:
    file.write(METRICS_DIR)

In [162]:
df_metrics = pd.merge(left=df_metrics, right=df_endpoints, how='left', left_on='experiment_name', right_on='experiment_name')
df_metrics.head()

# Convert df_metrics to CSV and write to S3
csv_buffer = io.StringIO()
df_metrics.to_csv(csv_buffer, index=False)
csv_data_metrics = csv_buffer.getvalue()
metrics_file_name = config['results']['all_metrics_file'].format(datetime=date_time)
metrics_s3_path = os.path.join(METRICS_DIR, metrics_file_name)
logger.info(f"results s3 path for metrics csv --> {metrics_s3_path}")
write_to_s3(csv_data_metrics, config['aws']['bucket'], "", METRICS_DIR, metrics_file_name)
logger.info(f"saved metrics results dataframe of shape={df_metrics.shape} in s3://{config['aws']['bucket']}/{metrics_s3_path}")

[2024-01-29 19:54:17,729] p58222 {234513869.py:10} INFO - results s3 path for metrics csv --> data/metrics/2024/01/29/19/52/mistral-7b-tgi-g5-v1/all_metrics.csv


[2024-01-29 19:54:17,955] p58222 {234513869.py:12} INFO - saved metrics results dataframe of shape=(17, 31) in s3://fmbt/data/metrics/2024/01/29/19/52/mistral-7b-tgi-g5-v1/all_metrics.csv
